In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# %cd /content/drive/My\ Drive/hw4
# !ls

/content/drive/My Drive/hw4
data.zip       testing_data.txt    training_nolabel.txt
hw4-RNN.ipynb  training_label.txt


In [ ]:
# !gdown --id '1lz0Wtwxsh5YCPdqQ3E3l_nbfJT1N13V8' --output data.zip
# !unzip data.zip
# !ls

Downloading...
From: https://drive.google.com/uc?id=1lz0Wtwxsh5YCPdqQ3E3l_nbfJT1N13V8
To: /content/drive/My Drive/hw4/data.zip
45.1MB [00:00, 96.9MB/s]
Archive:  data.zip
  inflating: training_label.txt      
  inflating: testing_data.txt        
  inflating: training_nolabel.txt    
data.zip       testing_data.txt    training_nolabel.txt
hw4-RNN.ipynb  training_label.txt


#### 1. 训练词向量
使用train.txt和test.txt训练word2vec，存储到‘w2v.model’

In [3]:
import os
import numpy as np
import torch
from gensim.models import word2vec
from torch.utils import data

In [ ]:
with open('training_label.txt', 'r') as f:
    for raw in f.readlines()[:5]:
        print(raw)
f.close()

1 +++$+++ are wtf ... awww thanks !

1 +++$+++ leavingg to wait for kaysie to arrive myspacin itt for now ilmmthek .!

0 +++$+++ i wish i could go and see duffy when she comes to mamaia romania .

1 +++$+++ i know eep ! i can ' t wait for one more day ....

0 +++$+++ so scared and feeling sick . fuck ! hope someone at hr help ... wish it would be wendita or karen .



In [ ]:
with open('testing_data.txt', 'r') as f:
    for raw in f.readlines()[:5]:
        print(raw)
f.close()

In [ ]:
raws = raw.strip('\n').split(',')
x = ','.join(raws[1:])
x = x.split(' ')
x

In [ ]:
def load_train_data(path):
    x = []
    y = []
    with open(path, 'r') as f:
        for raw in f.readlines():
            raws = raw.strip('\n').split(' ')
            y.append(int(raws[0]))
            x.append(raws[2:])
    f.close()
    return x, y

In [ ]:
def load_test_data(path):
    x = []
    with open(path, 'r') as f:
        for raw in f.readlines()[1:]:
            raws = raw.strip('\n').split(',')
            text = ','.join(raws[1:])
            x.append(text.split(' '))
    f.close()
    return x

In [ ]:
def train_word2vec(x):
    model = word2vec.Word2Vec(x, size=200, window=5, min_count=5, iter=10, sg=1)
    return model

In [ ]:
print('loading training data ……')
train_x, train_y = load_train_data('training_label.txt')
print('train_x.size: {}'.format(len(train_x)))

print('loading testing data……')
test_x = load_test_data('testing_data.txt')
print('test_x.size: {}'.format(len(test_x)))

word2vec_model = train_word2vec(train_x + test_x)
print('saving model ……')
word2vec_model.save('w2v.model')

loading training data……
train_x.size: 200000
loading testing data……
test_x.size: 200000
saving model ……


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


#### 2. 数据预处理

In [1]:
tmp = word2vec.Word2Vec.load('w2v.model')
tmp.vector_size
tmp['is']
tmp.wv.vocab

NameError: name 'word2vec' is not defined

In [ ]:
from torch import nn
from gensim.models import Word2Vec
class Preprocess():
    def __init__(self, sentences, seq_len, w2v_path='w2v.model'):
        self.seq_len = seq_len
        self.idx2word = [] # idx2word[0] = 'he'
        self.word2idx = {} # word2idx['he'] = 0
        self.embedding_matrix = [] # sentences的embedding矩阵
        self.embedding = Word2Vec.load(w2v_path) # 加载预训练好的词向量 embedding['word'] = vector
    
    def add_embedding(self, word):
        # 将<unk>,<pad>加入,随机初始化值即可
        vector = torch.randn(1, self.embedding.vector_size)
        self.word2idx[word] = len(self.idx2word)
        self.idx2word.append(word)
        self.embedding_matrix = torch.cat([self.embedding_matrix, vector], 0)

    def make_embedding(self):
        print('get embedding')
        for i, word in enumerate(self.embedding.wv.vocab):
            self.word2idx['word'] = len(self.word2idx)
            self.idx2word.append(word)
            self.embedding_matrix.append(self.embedding[word])
        
        self.embedding_matrix = torch.tensor(self.embedding_matrix)
        self.add_embedding('<PAD>')
        self.add_embedding('<UNK>')
        return self.embedding_matrix
    
    def pad_sequence(self, sentence): 
        if len(sentence) > self.seq_len:
            sentence = sentence[:seq_len]
        else:
            pad_len = self.seq_len - len(sentence)
            for _ in range(pad_len):
                sentence.append(self.word2idx['<PAD>'])
        assert len(sentence) == self.seq_len
        return sentence
      
    def sentence_word2idx(self, sentence):
        # 把句子里的词对应到index
        sentence_idx = []
        for i, word in enumerate(sentence):
            if word in self.word2idx.keys():
                sentence_idx.append(self.word2idx[word])
            else:
                sentence_idx.append(self.word2idx['<UNK>'])
        sentence_idx = self.pad_sequence(sentence_idx)
        return torch.tensor(sentence_idx)
              

#### 3.Dataset

In [ ]:
class TwitterDataset(data.Dataset):
    def __init__(self, x, y):
        self.data = x
        self.label = y
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        if self.label is not None:
            return self.data[index],self.label[index]
        else:
            return self.data[index]